In [1]:
from draftbot_sim_adapted import *
import sqlite3
import pandas as pd
import numpy as np
import json
import os

In [2]:
draft_path = 'C:/Users/trist/mtg_research/test_draft.txt'
cards_path = 'C:/Users/trist/mtg_research/mtg_draftbot/data/m20/m20-cards.json'
weights_path = 'C:/Users/trist/mtg_research/mtg_draftbot/data/new_generated_weights.json'

In [3]:
def comp_generator(num_iter,agents,n_cards_in_pack,rounds,cards_path,card_values_path,packs_input_file):
    '''Streamline comparison of drafts and create outputs with player prefs;
    this gives us quick glances into proxies for how '''

    n_drafters = len(agents)
    ss_prefs = []
    ms_prefs = []

    r = 1

    for r in range(1,num_iter+1):
          draft_ss = Draft(n_drafters=n_drafters,
                    n_rounds=rounds,
                    n_cards_in_pack=n_cards_in_pack,
                    cards_path=cards_path,
                    card_values_path=card_values_path)

    draft_ms = MultiStratDraft(
        n_rounds=3,
        n_cards_in_pack=14,
        cards_path=cards_path,
        card_values_path=weights_path,
        packs_input_file=draft_path,
        agent_list=agents)
                    

    draft_ss.draft()
    result_array = np.empty(0)
     #result_array = np.append(result_array,np.arange(0,draft_ss.n_drafters))
    for item,idx in enumerate(draft_ss.drafter_preferences):
          player=np.append(item,r)
          player=np.append(player,idx)
          result_array = np.append(result_array,player)
    ss_prefs.append(result_array.reshape((8,12)))

    if os.path.exists(packs_input_file):
               os.remove(packs_input_file)
    save_packs_array(draft_ss,packs_input_file)

    draft_ms.draft()
    result_array = np.empty(0)
    for item,idx in enumerate(draft_ms.drafter_preferences):
          player=np.append(item,r)
          player=np.append(player,idx)
          result_array = np.append(result_array,player)

    ms_prefs.append(result_array.reshape((8,12)))
    r += 1


    return (ms_prefs,ss_prefs)

def macro_prefs_sum_stats(draft_comp):
    '''Take a draft comp and output results'''

    ss_prefs = draft_comp[1]
    ms_prefs = draft_comp[0]
    ss_prefs_df = pd.DataFrame(np.concatenate(ss_prefs))
    ms_prefs_df = pd.DataFrame(np.concatenate(ms_prefs))

    ssdf = ss_prefs_df.set_index([0,1])
    ssdf = ssdf.groupby(by=[0]).mean()
    ssdf['maximum'] = ssdf.max(axis=1)
    ssdf['minimum'] = ssdf.min(axis=1)
    ssdf['average'] = ssdf.mean(axis=1)
    ssdf['stdev'] = ssdf.std(axis=1)

    msdf = ms_prefs_df.set_index([0,1])
    msdf = msdf.groupby(by=[0]).mean()
    msdf['maximum'] = msdf.max(axis=1)
    msdf['minimum'] = msdf.min(axis=1)
    msdf['average'] = msdf.mean(axis=1)
    msdf['stdev'] = msdf.std(axis=1)

    return (msdf,ssdf)

In [4]:
agents_onek = [
  Naive_Pass_Agent.decision_function, 
  Basic_Agent.decision_function, 
  Basic_Agent.decision_function,
  Basic_Agent.decision_function,
  Basic_Agent.decision_function,
  Basic_Agent.decision_function,
  Basic_Agent.decision_function,
  Basic_Agent.decision_function, 
]

one_k_comp = comp_generator(1000,agents_onek,14,3,cards_path,weights_path,
draft_path)

agents_twosix = [
  Naive_Pass_Agent.decision_function, 
  Naive_Pass_Agent.decision_function, 
  Basic_Agent.decision_function,
  Basic_Agent.decision_function,
  Basic_Agent.decision_function,
  Basic_Agent.decision_function,
  Basic_Agent.decision_function,
  Basic_Agent.decision_function, 
]

one_k_twosix = comp_generator(1000,agents_twosix,14,3,cards_path,weights_path,
draft_path)

agents_threefive = [
  Naive_Pass_Agent.decision_function, 
  Naive_Pass_Agent.decision_function, 
  Naive_Pass_Agent.decision_function,
  Basic_Agent.decision_function,
  Basic_Agent.decision_function,
  Basic_Agent.decision_function,
  Basic_Agent.decision_function,
  Basic_Agent.decision_function, 
]

threefive_k = comp_generator(1000,agents_threefive,14,3,cards_path,weights_path,
draft_path)

agents_fourfour = [
  Naive_Pass_Agent.decision_function, 
  Naive_Pass_Agent.decision_function, 
  Naive_Pass_Agent.decision_function,
  Naive_Pass_Agent.decision_function,
  Basic_Agent.decision_function,
  Basic_Agent.decision_function,
  Basic_Agent.decision_function,
  Basic_Agent.decision_function, 
]
fourfour_k = comp_generator(1000,agents_fourfour,14,3,cards_path,weights_path,
draft_path)

In [8]:
thousand_one = macro_prefs_sum_stats(one_k_comp)
thousand_two = macro_prefs_sum_stats(one_k_twosix)
thousand_three = macro_prefs_sum_stats(threefive_k)
thousand_four = macro_prefs_sum_stats(fourfour_k)

In [9]:
#Difference in maxes is largest for the position with the new strategy
#Note that a negative reduction here would be analogous to saying that a preference toward the most preferred archetype goes down
#That really means that drafter 0 has a more diverse/less specialized pool because they don't have as strong a favored archetype as the others. 

print(thousand_one[0]['average'] - thousand_one[1]['average'])
print(thousand_two[0]['average'] - thousand_two[1]['average'])

0
0.0   -0.086229
1.0    0.189274
2.0    0.151954
3.0   -0.021360
4.0    0.049656
5.0   -0.035701
6.0    0.090813
7.0    0.000000
Name: average, dtype: float64
0
0.0   -0.865086
1.0   -0.564272
2.0    0.334897
3.0    0.256724
4.0    0.049261
5.0   -0.148265
6.0    0.090156
7.0    0.078418
Name: average, dtype: float64


In [10]:
print(thousand_three[0]['average'] - thousand_three[1]['average'])
print(thousand_four[0]['average'] - thousand_four[1]['average'])

0
0.0   -0.213447
1.0   -0.393570
2.0   -0.342964
3.0    0.801723
4.0    0.184000
5.0    0.054787
6.0   -0.099165
7.0   -0.080231
Name: average, dtype: float64
0
0.0   -0.563759
1.0   -0.663086
2.0   -0.496640
3.0   -0.260630
4.0    1.138149
5.0    0.520829
6.0    0.178021
7.0    0.199807
Name: average, dtype: float64
